In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress unnecessary warnings
import numpy as np
from os import listdir
import re

import sqlalchemy as sa
import cx_Oracle

from pandas.tseries.offsets import Day, MonthEnd
from dateutil.relativedelta import relativedelta
import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio.plotly import plot, iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
zone = pd.read_csv('taxi_zones.csv')
zone.columns = zone.columns.str.lower()
zone.head(2)

,locationid,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone


In [3]:
pu_zone = zone.copy()
pu_zone.columns = 'pu'+pu_zone.columns

do_zone = zone.copy()
do_zone.columns = 'do'+do_zone.columns

In [4]:
cal = pd.read_csv('454_calendar.csv')
cal.columns = cal.columns.str.lower()
cal.head(2)

,date,fiscalyear,fiscalquarter,fiscalmonthnumber,fiscalmonthofquarter,fiscalweekofyear,dayofweek,fiscalmonthname,fiscalmonthyear,fiscalquarteryear,dayofmonthnumber,dayname
0,2017-02-05,2017,1,1,1,1,0,February,17-Feb,12017,5,Sunday
1,2017-02-06,2017,1,1,1,1,1,February,17-Feb,12017,6,Monday


In [5]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  1456 non-null   object
 1   fiscalyear            1456 non-null   int64 
 2   fiscalquarter         1456 non-null   int64 
 3   fiscalmonthnumber     1456 non-null   int64 
 4   fiscalmonthofquarter  1456 non-null   int64 
 5   fiscalweekofyear      1456 non-null   int64 
 6   dayofweek             1456 non-null   int64 
 7   fiscalmonthname       1456 non-null   object
 8   fiscalmonthyear       1456 non-null   object
 9   fiscalquarteryear     1456 non-null   int64 
 10  dayofmonthnumber      1456 non-null   int64 
 11  dayname               1456 non-null   object
dtypes: int64(8), object(4)
memory usage: 136.6+ KB


In [6]:
#getting the list of the files in the current directory
files = os.listdir()
#finding the files that ended with "_taxi_trips.csv"
files = [f for f in files if f[-15:]=='_taxi_trips.csv']

In [7]:
trips = pd.DataFrame()
for f in files:
    temp = pd.read_csv(f)
    #extracting date only
    temp['date'] = temp['lpep_pickup_datetime'].str.slice(0,10)
    
    #converting the object to datetime format
    temp['lpep_pickup_datetime'] = pd.to_datetime(temp['lpep_pickup_datetime'])
    temp['lpep_dropoff_datetime'] = pd.to_datetime(temp['lpep_dropoff_datetime'])
    
    #not sent via store & forward
    temp = temp.loc[temp['store_and_fwd_flag']=='N'] 
    #street-hailed trips only
    temp = temp.loc[temp['trip_type']==1] 
    #standard-rated trips only
    temp = temp.loc[temp['RatecodeID']==1] 
    #card or cash payment only
    temp = temp.loc[(temp['payment_type']==1)|(temp['payment_type']==2)] 
    #2017 to 2021 trips only
    temp = temp.loc[(temp['lpep_pickup_datetime']<='2020-12-31')&(temp['lpep_pickup_datetime']>='2017-01-01')]
    #swapping pickup and dropoff datetime 
    temp['pickup_time'] = np.where(((temp['lpep_dropoff_datetime']-temp['lpep_pickup_datetime'])/np.timedelta64(1,'h')) < 0, 
                                    temp['lpep_dropoff_datetime'],temp['lpep_pickup_datetime']) 
    temp['dropoff_time'] = np.where(((temp['lpep_dropoff_datetime']-temp['lpep_pickup_datetime'])/np.timedelta64(1,'h')) < 0, 
                                     temp['lpep_pickup_datetime'],temp['lpep_dropoff_datetime'])
    #removing trips that lasted more than 24 hours
    temp = temp[~(((temp['dropoff_time']-temp['pickup_time'])/np.timedelta64(1,'h')) > 24)]
    
    #removing those trips with trip disntance and fare amount of zero
    temp = temp[~((temp['trip_distance']==0)&(temp['fare_amount']==0))] 
    
    #making the positive charges
    temp['fare_amount'] = temp['fare_amount'].abs()
    temp['extra'] = temp['extra'].abs()
    temp['mta_tax'] = temp['mta_tax'].abs()
    
    #calculating distance when there's fare amount 
    temp['trip_distance'] = np.where((temp['fare_amount']>0)&(temp['trip_distance']==0), 
                                     (temp['fare_amount']-2.5)/2.5,
                                     temp['trip_distance']) 
    
    #calculating fare when there's distance 
    temp['fare_amount'] = np.where((temp['trip_distance']>0)&(temp['fare_amount']==0), 
                                   (temp['trip_distance']*2.5)+2.5,
                                   temp['fare_amount'])
    
    #removing the columns with limited variables filtered above
    temp = temp.drop(columns=['store_and_fwd_flag','payment_type','RatecodeID','trip_type',
                              'lpep_dropoff_datetime','lpep_pickup_datetime','total_amount'])
    temp.columns = temp.columns.str.lower()
    trips = pd.concat([temp,trips])

In [8]:
trips = trips.merge(pu_zone,on='pulocationid',how='left')
trips = trips.merge(do_zone,on='dolocationid',how='left')

In [9]:
#removing the pickup from unknown zone
trips = trips.dropna(subset=['puzone'])
trips = trips.loc[trips['puborough']!='Unknown']

#removing the dropoff to unknown zone
trips = trips.dropna(subset=['dozone'])
trips = trips.loc[trips['doborough']!='Unknown']

In [46]:
trips['year'] = trips['pickup_time'].dt.to_period("Y")

In [47]:
trips['month'] = trips['pickup_time'].dt.strftime("%B")

In [45]:
trips['week'] = trips['pickup_time'].dt.strftime("%V")

In [49]:
trips['day_of_week_no'] = trips['pickup_time'].dt.dayofweek
trips['day_of_week'] = trips['pickup_time'].dt.day_name()

In [50]:
trips['hour'] = trips['pickup_time'].dt.strftime("%H")

In [51]:
trips.head()

,vendorid,pulocationid,dolocationid,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,doborough,dozone,doservice_zone,week,year,month,month_no,day_of_week_no,day_of_week,hour
0,2.0,74,75,1.0,1.47,6.5,0.0,0.5,0.0,0.0,...,Manhattan,East Harlem South,Boro Zone,01,2020,January,01,2,Wednesday,06
1,2.0,74,75,1.0,1.49,6.5,0.0,0.5,0.0,0.0,...,Manhattan,East Harlem South,Boro Zone,01,2020,January,01,2,Wednesday,13
2,2.0,74,75,1.0,1.31,6.5,0.0,0.5,0.0,0.0,...,Manhattan,East Harlem South,Boro Zone,01,2020,January,01,2,Wednesday,14
3,2.0,74,75,1.0,1.43,6.5,0.0,0.5,0.0,0.0,...,Manhattan,East Harlem South,Boro Zone,01,2020,January,01,3,Thursday,06
4,2.0,74,75,1.0,1.10,6.5,0.0,0.5,0.0,0.0,...,Manhattan,East Harlem South,Boro Zone,01,2020,January,01,3,Thursday,09


# Plotting

## Drop-off locations 

In [84]:
df = trips.groupby(['doborough','dozone']).agg({'pickup_time':'count'}).reset_index()
fig = px.sunburst(df.reset_index(), path=['doborough', 'dozone'], values='pickup_time',color='doborough',
                  color_discrete_map={'Brooklyn':'#F3F0D7', 'Queens':'#949db9', 'Bronx':'#F7DBF0','Manhattan':'#CDF0EA'})

fig.update_layout(hovermode = 'x', 
                  title="drop off by zone", font=dict(size = 10, color = 'white'), 
                  plot_bgcolor='black', paper_bgcolor='black',font_family="Roboto", title_font_family="Roboto" )
fig.show()

## Pick-up locations

In [39]:
df = trips.groupby(['puborough','puzone']).agg({'pickup_time':'count'}).reset_index()
fig = px.sunburst(df.reset_index(), path=['puborough', 'puzone'], values='pickup_time',color='puborough',
                  color_discrete_map={'Manhattan':'#C9CCD5', 'Bronx':'#CDF0EA', 'Queens':'#D3E4CD','Brooklyn':'#FEF5ED'})
fig.update_layout( hovermode = 'x', 
                  title="pick up by zone", font=dict(size = 10, color = 'white'), 
                  plot_bgcolor='black', paper_bgcolor='black',font_family="Roboto", title_font_family="Roboto" )
fig.show()

## Busiest day and time 

In [117]:
fig = px.bar(time, x="pickup_time", y="hour", color='day_of_week', orientation='h',height=400,width=900,
             color_discrete_map={'Sunday':'#CDF0EA', 'Monday':'#F9CF93', 'Tuesday':'#ADC2A9','Wednesday':'#FEF5ED',
                                 'Thursday':'#C7B198', 'Friday':'#D3E4CD', 'Saturday':'#898B8A'})#949db9


fig.update_layout(hovermode = 'x', yaxis=dict(showgrid=False,showline=False,), xaxis=dict(showgrid=False,showline=False,),
                  title="time breakdown", font=dict(size = 10, color = 'white'), 
                  plot_bgcolor='black', paper_bgcolor='black',font_family="Roboto", title_font_family="Roboto") 
fig.show()

## 2020 Vs 2017 

In [58]:
df_scatter = trips.pivot_table(index='week',columns='year',values='fare_amount',aggfunc='sum')
df_scatter.columns = 'Y'+ df_scatter.columns.astype(str)
df_scatter = df_scatter.reset_index()
df_scatter.head(2)

year,week,Y2017,Y2018,Y2019,Y2020
0,01,2318619.435,1936238.625,1543389.770,525922.550
1,02,2683086.050,2173906.900,1674619.575,850166.145


In [118]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_scatter['week'],
    y=df_scatter['Y2020'],
    name='2020',
    marker=dict(
        color='rgba(156, 165, 196, 0.95)',
        line_color='rgba(156, 165, 196, 1.0)',
    )
))
fig.add_trace(go.Scatter(
    x=df_scatter['week'], y=df_scatter['Y2017'],
    name='2017',
    marker=dict(
        color='rgba(204, 204, 204, 0.95)',
        line_color='rgba(217, 217, 217, 1.0)'
    )
))

fig.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))



fig.update_layout(
    title="2017 Vs 2020",font_family="Roboto", title_font_family="Roboto" ,
    xaxis=dict(
        showgrid=False,
        showline=False,
        linecolor='black',
        tickfont_color='black',#'rgb(102, 102, 102)',
        showticklabels=False,
        dtick=10,
        ticks='outside',
        tickcolor='black',#'rgb(102, 102, 102)',
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
    ),
    margin=dict(l=140, r=40, b=50, t=80),
    legend=dict(
        font_size=10,
        yanchor='middle',
        xanchor='right',
    ),
    width=1000,
    height=250,
    paper_bgcolor='black',
    plot_bgcolor='black',
    hovermode='closest',
)
fig.show()

# At a glance

In [142]:
print("What's the average number of trips we can expect this week? - {0:.0f}".format(trips.loc[trips['year']=='2020'].groupby(['week']).agg({'pickup_time':'count'}).reset_index().pickup_time.mean()))
print("What's the average fare per trip we expect to collect? - ${0:.0f}".format(trips.fare_amount.mean()))
print("What's the average distance traveled per trip? - {0:.0f} miles".format(trips.trip_distance.mean()))

What's the average number of trips we can expect this week? - 21614
What's the average fare per trip we expect to collect? - $12
What's the average distance traveled per trip? - 3 miles


# Appendix

In [92]:
do = trips.groupby(['doborough','dozone']).agg({'pickup_time':'count'}).sort_values(by=['pickup_time'],ascending=False)
do['percent'] = np.round((do['pickup_time']/do.pickup_time.sum()*100),decimals=1)
do.head(5)

pickup_time  percent
doborough dozone                                    
Manhattan East Harlem North          955946      3.6
          Central Harlem North       915492      3.5
          Central Harlem             825188      3.1
Queens    Astoria                    797293      3.0
          Jackson Heights            727796      2.8

In [93]:
pu = trips.groupby(['puborough','puzone']).agg({'pickup_time':'count'}).sort_values(by=['pickup_time'],ascending=False)
pu['percent'] = np.round((pu['pickup_time']/pu.pickup_time.sum()*100),decimals=1)
pu.head(5)

pickup_time  percent
puborough puzone                                 
Manhattan East Harlem North      1833599      7.0
          East Harlem South      1584533      6.0
          Central Harlem         1535265      5.9
Queens    Astoria                1277025      4.9
          Elmhurst               1172017      4.5